In [1]:
import pandas as pd 
import numpy as np 
import time 
import heapq as hq 

Data Importing

In [2]:
KNN_Train = pd.read_csv('TrainData.csv') 
KNN_Test = pd.read_csv('TestData.csv') 

In [3]:
KNN_Train.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,y
0,0.042795,5,-1.06270,0,1.12890,2,8,3,0,1,0.14609,-0.21858,-0.077733,0,-1
1,0.880270,1,-1.00790,0,1.12890,0,4,2,0,1,-0.14744,-0.21858,-2.331500,0,-1
2,-0.033339,0,0.24469,3,-0.43973,1,6,3,0,1,-0.14744,-0.21858,-0.077733,0,-1
3,1.108700,0,0.42523,2,-1.22400,0,6,2,4,1,-0.14744,-0.21858,-0.077733,0,-1
4,-0.794680,0,1.40660,0,1.12890,0,5,0,4,0,-0.14744,-0.21858,-0.077733,12,-1


In [4]:
KNN_Test.head()

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,y
0,-1.029000,0,0.352090,2,-1.216500,2,7,1,4,1,-0.14543,-0.21916,-0.078886,0,-1
1,-0.057421,0,-0.944960,3,-0.434880,0,9,2,0,1,-0.14543,-0.21916,0.750110,0,-1
2,-0.804770,4,1.395000,5,0.737570,0,12,2,0,1,-0.14543,-0.21916,-0.078886,0,1
3,0.390990,0,-0.277360,1,-0.044064,0,7,2,4,1,0.85260,-0.21916,-0.078886,0,1
4,-0.356360,0,0.085941,12,-1.607300,2,2,3,0,1,-0.14543,-0.21916,-0.907880,0,-1


In [5]:
KNN_Train.shape

(16384, 15)

In [6]:
KNN_Test.shape

(5000, 15)

In [7]:
X = KNN_Train.drop('y', axis=1)
y = KNN_Train.y
KNN_Test_X = KNN_Test.drop('y',axis=1)
KNN_Test_y = KNN_Test.y

Euclidean distance between x1 and x2

In [8]:

def distance(x1, x2):
    distance = np.sqrt(np.sum([(a-b)*(a-b) for a, b in zip(x1, x2)]))  
    
    return distance



distance(x1=X.iloc[0], x2=X.iloc[4])

14.658955171720972

In [9]:


test_pt = KNN_Test_X.iloc[0]


distances = []

for i in X.index:
    
    distances.append(distance(test_pt, X.iloc[i]))
    
df_dists = pd.DataFrame(data=distances, index=X.index, columns=['E.D. Between first row in Test and all rows in Train'])
df_dists.head()

,E.D. Between first row in Test and all rows in Train
0,7.663959
1,7.146332
2,4.960469
3,3.251949
4,12.832295


In [10]:


df_nn = df_dists.sort_values(by=['E.D. Between first row in Test and all rows in Train'], axis=0)[:5]
df_nn

,E.D. Between first row in Test and all rows in Train
8874,1.433690
4549,1.523204
3060,1.623624
1669,1.672579
1720,1.678539


In [11]:
from collections import Counter
counter = Counter(y[df_nn.index])
counter.most_common()[0][0]

-1

In [12]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,random_state=123)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
def knn_predict(X_train, X_test, y_train, y_test, k, p):
    
    # Counter will assit with label voting
    from collections import Counter
    
    # Make 1 prediction per test datapoint
    y_hat_test = []

    for test_point in X_test:
        distances = []
        for train_point in X_train:
            distance_t = distance(test_point, train_point)
            distances.append(distance_t)
        
        # Store Distances in a Dataframe
        df_dists = pd.DataFrame(data=distances, columns=['dist'], 
                                index=y_train.index)
        
        # Sort distances with considerations to k neighbours
        df_nn = df_dists.sort_values(by=['dist'], axis=0)[:k]

        # Create counter to track labels
        counter = Counter(y_train[df_nn.index])

        # Find most common label
        prediction = counter.most_common()[0][0]
        
        # Append prediction to output list
        y_hat_test.append(prediction)
        
    return y_hat_test 

In [14]:

y_hat_test_10_2 = knn_predict(X_train, X_test, y_train, y_test, k=10, p=2)
acc_10_2 = np.sum(np.equal(y_test, y_hat_test_10_2)) / len(y_test)
acc_10_2


0.8193469636862984

In [15]:
y_hat_test_20_2 = knn_predict(X_train, X_test, y_train, y_test, k=20, p=2)
acc_20_2 = np.sum(np.equal(y_test, y_hat_test_20_2)) / len(y_test)
acc_20_2

0.8245346353371986

In [16]:
y_hat_test_50_2 = knn_predict(X_train, X_test, y_train, y_test, k=50, p=2)
acc_50_2 = np.sum(np.equal(y_test, y_hat_test_50_2)) / len(y_test)
acc_50_2

0.8190418065303632

In [17]:
y_hat_test_100_2 = knn_predict(X_train, X_test, y_train, y_test, k=100, p=2)
acc_100_2 = np.sum(np.equal(y_test, y_hat_test_100_2)) / len(y_test)
acc_100_2

0.8123283490997864

In [51]:
print("Accuracy is",acc_20_2*100)

Accuracy is 82.45346353371986
